In [11]:
#cross correlation for every residue in a chain 
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import RMSD

def calculate_covariance_matrix(coordinates):
    N = coordinates.shape[0] // 3
    print("ok")
    X = coordinates.reshape(N, 3, N, 3)
    X_avg = np.mean(X, axis=0)
    X_fluct = X - X_avg
    C = np.einsum('aijb,aikb->jk', X_fluct, X_fluct) / N
    return C

def perform_pca(covariance_matrix):
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]
    return eigenvalues, eigenvectors

def construct_dccm(eigenvalues, eigenvectors):
    num_modes = min(20, eigenvalues.shape[0])
    dccm = np.zeros_like(eigenvectors.dot(eigenvectors.T))
    for i in range(num_modes):
        mode_contribution = eigenvalues[i] * np.outer(eigenvectors[:, i], eigenvectors[:, i])
        dccm += mode_contribution
    return dccm



In [34]:
# Path to the PDB and trajectory files
pdb_file = '5uh5.pdb'
xtc_file = '5uh5.xtc'

# Load PDB and trajectory using MDAnalysis
u = mda.Universe(pdb_file, xtc_file)

# Select alpha-carbon atoms
selection = u.select_atoms('name CA')

# Align trajectory to the reference structure (optional)
ref = u.select_atoms('name CA').positions
alignment = align.AlignTraj(u, u, select="name CA", in_memory=True).run()
rmsd = RMSD(u, select="name CA", ref=ref).run()

# Extract alpha-carbon coordinates
coordinates = selection.positions.flatten()




: 

: 

In [ ]:
print("rmsd")

rmsd


In [15]:
# Calculate covariance matrix
print(coordinates)
covariance_matrix = calculate_covariance_matrix(coordinates)


[104.94      93.       171.33002  ... 101.630005  59.800003  94.86    ]
ok


ValueError: cannot reshape array of size 9723 into shape (3241,3,3241,3)

In [ ]:

# Perform PCA
eigenvalues, eigenvectors = perform_pca(covariance_matrix)



In [ ]:
# Construct DCCM
dccm = construct_dccm(eigenvalues, eigenvectors)